<a href="https://colab.research.google.com/github/Gribouille-91200/OpenclassroomsProject/blob/main/Copie_de_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Python 3.11.11
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp13gp27d9".


In [24]:
%%writefile hello.cu
#include <stdio.h>

__global__ void hello(){
printf("Bonjour du bloc : %u, thread : %u\n", blockIdx.x, threadIdx.x);
}

int main(){
hello<<< 2 , 2 >>>();
cudaDeviceSynchronize();
}


Overwriting hello.cu


In [20]:

%%writefile hello.cu

#include<stdio.h>
__global__ void hello(void)
{
    printf("GPU: Hello!\n");
}
int main(int argc,char **argv)
{
    printf("CPU: Hello!\n");
    hello<<<1,10>>>();
    cudaDeviceReset();
    return 0;
}

Writing hello.cu


In [ ]:
!nvidia-smi


Wed Feb 26 21:02:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [25]:
!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 hello.cu -o hello

nvcc fatal   : Value 'sm_37' is not defined for option 'gpu-architecture'


In [26]:
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 hello.cu -o hello

In [27]:
!./hello

Bonjour du bloc : 1, thread : 0
Bonjour du bloc : 1, thread : 1
Bonjour du bloc : 0, thread : 0
Bonjour du bloc : 0, thread : 1


In [ ]:
%%writefile hello.cu

#include <stdio.h>
 __global__ void hello(void){
printf("Bonjour du bloc : %u, thread : %u\n", blockIdx.x, threadIdx.x);
}
int main(int argc,char **argv){
hello<<< 2 , 2 >>>();
cudaDeviceSynchronize();
}

Overwriting hello.cu


In [ ]:
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 hello.cu -o hello

In [ ]:
!./hello

Bonjour du bloc : 1, thread : 0
Bonjour du bloc : 1, thread : 1
Bonjour du bloc : 0, thread : 0
Bonjour du bloc : 0, thread : 1


In [122]:
%%writefile vector_add.cu
#include <stdio.h>
#include<stdlib.h>
#define N 100000

__global__ void vector_add(float *out, float *a, float *b, int n) {
       int id = (blockIdx.x * blockDim.x) + threadIdx.x;
       //printf("Bonjour du bloc : %u, thread : %u\n", blockIdx.x, threadIdx.x);

    // Make sure we do not go out of bounds;
    // Threads allocated could be larger than array length
    if (id < n){
        out[id] = a[id] + b[id];
    }
;
}

int main(){
    float *a, *b, *out;
    float *d_a, *d_b, *d_out;

    a = (float*)malloc(sizeof(float) * N);
    b = (float*)malloc(sizeof(float) * N);
    out = (float*)malloc(sizeof(float) * N);

        // Initialize array
    for(int i = 0; i < N; i++){
        a[i] = 1.0f; b[i] = 2.0f;
    }

    // Allocate device memory for a
    cudaMalloc((void**)&d_a, sizeof(float) * N);
    cudaMalloc((void**)&d_b, sizeof(float) * N);
    cudaMalloc((void**)&d_out, sizeof(float) * N);

    // Transfer data from host to device memory
    cudaMemcpy(d_a, a, sizeof(float) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(float) * N, cudaMemcpyHostToDevice);

    // Default number of threads in each thread block
    int blockSize = 256;

    // Number of thread blocks in grid needs to be based on array size
    // and block size
    int gridSize = (int)ceil((float)N/blockSize);

    printf("add vectors on device using grid with ");
    printf("%d blocks of %d threads each.\n", gridSize, blockSize);

    vector_add<<<gridSize,blockSize>>>(d_out, d_a, d_b, N);

   // Copy data from device array d_C to host array C
    cudaMemcpy(out, d_out, sizeof(float) * N, cudaMemcpyDeviceToHost);

    /*for (int i=0; i< N; i++){
      printf("%f +%f  = %f \n", a[i],b[i], out[i]);
    }*/


    // Cleanup after kernel execution
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);

    free(a);
    free(b);
    free(out);
}

Overwriting vector_add.cu


In [123]:
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 vector_add.cu -o vector_add

In [124]:
!./vector_add

add vectors on device using grid with 391 blocks of 256 threads each.
